## 1. Installation & Configuration

In [21]:
import pandas as pd
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliothèques chargées")

✓ Bibliothèques chargées


In [22]:
import os
from dotenv import load_dotenv
from pathlib import Path
load_dotenv(Path(".env"))
newsapi = NewsApiClient(api_key=os.getenv("NEWSAPI_API_KEY"))

# Device (CPU ou GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


## 2. Chargement de votre modèle FinBERT

In [23]:
# Charger votre modèle fine-tuné
MODEL_PATH = "./news_finbert_sentiment_model"

print("Chargement du modèle FinBERT...")
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=2,
    use_safetensors=True
)
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
print("✓ Modèle FinBERT chargé et prêt")

Chargement du modèle FinBERT...
✓ Modèle FinBERT chargé et prêt


## 3. Fonction de récupération de news

In [24]:
def fetch_financial_news(query="finance OR stocks OR market", 
                         country=None, 
                         language="en",
                         page_size=20,
                         days_back=1):
    """
    Récupère des news financières via NewsAPI
    
    Args:
        query: Mots-clés (ex: "Apple OR Tesla OR Bitcoin")
        country: Code pays ISO (ex: "us", "fr", "gb") - None pour tous
        language: Code langue ("en", "fr", etc.)
        page_size: Nombre d'articles (max 100)
        days_back: Nombre de jours en arrière
    
    Returns:
        Liste de dictionnaires avec les news
    """
    
    # Date de début
    from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
    
    try:
        # Recherche d'articles
        articles = newsapi.get_everything(
            q=query,
            language=language,
            from_param=from_date,
            sort_by='publishedAt',
            page_size=page_size
        )
        
        # Extraction des données pertinentes
        news_list = []
        for article in articles.get('articles', []):
            news_list.append({
                'title': article.get('title', ''),
                'description': article.get('description', ''),
                'content': article.get('content', ''),
                'source': article.get('source', {}).get('name', 'Unknown'),
                'url': article.get('url', ''),
                'published_at': article.get('publishedAt', ''),
                'author': article.get('author', 'Unknown')
            })
        
        print(f"✓ {len(news_list)} articles récupérés")
        return news_list
    
    except Exception as e:
        print(f"❌ Erreur API: {e}")
        return []

## 4. Fonction d'analyse de sentiment

In [25]:
def analyze_sentiment(text, model, tokenizer, device, max_length=512):
    """
    Analyse le sentiment d'un texte avec votre modèle FinBERT
    
    Args:
        text: Texte à analyser
        model: Modèle FinBERT
        tokenizer: Tokenizer
        device: CPU ou GPU
        max_length: Longueur max (512 pour news)
    
    Returns:
        dict avec sentiment, confiance, probabilités
    """
    
    if not text or len(text.strip()) == 0:
        return {
            'sentiment': 'Unknown',
            'confidence': 0.0,
            'prob_negative': 0.5,
            'prob_positive': 0.5
        }
    
    # Tokenization
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)[0]
        
        prediction = torch.argmax(logits, dim=1).item()
        confidence = probabilities[prediction].item()
    
    sentiment_label = "Positive" if prediction == 1 else "Negative"
    
    return {
        'sentiment': sentiment_label,
        'confidence': confidence,
        'prob_negative': probabilities[0].item(),
        'prob_positive': probabilities[1].item()
    }

## 5. Pipeline complet : Fetch + Analyze

In [26]:
def analyze_news_pipeline(query="finance", country=None, page_size=10):
    """
    Pipeline complet : récupère les news et analyse le sentiment
    
    Returns:
        DataFrame avec news + analyse de sentiment
    """
    
    print("="*80)
    print("🔄 DÉBUT DU PIPELINE")
    print("="*80)
    
    # 1. Récupérer les news
    print(f"\n📡 Récupération des news (query='{query}')...")
    news_list = fetch_financial_news(query=query, country=country, page_size=page_size)
    
    if not news_list:
        print("❌ Aucune news récupérée")
        return pd.DataFrame()
    
    # 2. Analyser chaque news
    print(f"\n🤖 Analyse de sentiment en cours...")
    results = []
    
    for i, news in enumerate(news_list, 1):
        # Combiner titre + description pour l'analyse
        full_text = f"{news['title']} {news['description']}"
        
        # Analyser le sentiment
        sentiment_result = analyze_sentiment(full_text, model, tokenizer, device)
        
        # Combiner les données
        results.append({
            'title': news['title'],
            'source': news['source'],
            'published_at': news['published_at'],
            'sentiment': sentiment_result['sentiment'],
            'confidence': sentiment_result['confidence'],
            'prob_negative': sentiment_result['prob_negative'],
            'prob_positive': sentiment_result['prob_positive'],
            'url': news['url']
        })
        
        # Affichage progression
        if i % 5 == 0:
            print(f"   Analysé {i}/{len(news_list)} articles...")
    
    # 3. Créer DataFrame
    df = pd.DataFrame(results)
    
    print(f"\n✓ {len(df)} news analysées avec succès")
    print("="*80)
    
    return df

## 🔍 DIAGNOSTIC : Vérification des labels du modèle

**Problème détecté** : Toutes les news sont classées "Negative". Vérifions si les labels sont inversés.

In [27]:
# Test 1 : News clairement positives vs négatives
test_cases_diagnostic = {
    "Positive": "Apple stock surges to record highs as earnings beat expectations. Investors celebrate strong revenue growth and optimistic guidance.",
    "Negative": "Company files for bankruptcy amid mounting debt crisis. Shareholders face massive losses as stock plummets 90%."
}

print("="*80)
print("🔬 TEST DIAGNOSTIC DES LABELS")
print("="*80)

for expected_sentiment, text in test_cases_diagnostic.items():
    result = analyze_sentiment(text, model, tokenizer, device)
    
    print(f"\n📝 Texte : {text[:80]}...")
    print(f"   ✓ Sentiment attendu  : {expected_sentiment}")
    print(f"   🤖 Sentiment prédit   : {result['sentiment']}")
    print(f"   📊 Prob Negative     : {result['prob_negative']:.2%}")
    print(f"   📊 Prob Positive     : {result['prob_positive']:.2%}")
    print(f"   🎯 Confiance         : {result['confidence']:.2%}")
    
    # Vérification
    if result['sentiment'] != expected_sentiment:
        print(f"   ⚠️  ERREUR : Labels probablement inversés!")
    else:
        print(f"   ✓ Correct")

print("\n" + "="*80)

🔬 TEST DIAGNOSTIC DES LABELS

📝 Texte : Apple stock surges to record highs as earnings beat expectations. Investors cele...
   ✓ Sentiment attendu  : Positive
   🤖 Sentiment prédit   : Positive
   📊 Prob Negative     : 21.25%
   📊 Prob Positive     : 78.75%
   🎯 Confiance         : 78.75%
   ✓ Correct

📝 Texte : Company files for bankruptcy amid mounting debt crisis. Shareholders face massiv...
   ✓ Sentiment attendu  : Negative
   🤖 Sentiment prédit   : Negative
   📊 Prob Negative     : 96.03%
   📊 Prob Positive     : 3.97%
   🎯 Confiance         : 96.03%
   ✓ Correct



## 🔧 SOLUTION : Fonction avec labels corrigés

Si le test ci-dessus montre que les labels sont inversés, utilisez cette version corrigée :

In [28]:
def analyze_sentiment_fixed(text, model, tokenizer, device, max_length=512):
    """
    Version CORRIGÉE avec labels inversés
    0 = Positive, 1 = Negative (inversé par rapport à l'entraînement)
    """
    
    if not text or len(text.strip()) == 0:
        return {
            'sentiment': 'Unknown',
            'confidence': 0.0,
            'prob_negative': 0.5,
            'prob_positive': 0.5
        }
    
    # Tokenization
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Prédiction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)[0]
        
        prediction = torch.argmax(logits, dim=1).item()
        confidence = probabilities[prediction].item()
    
    # 🔧 CORRECTION : Inverser les labels
    # Le modèle a appris : 0=Negative, 1=Positive
    # Mais on veut      : 0=Positive, 1=Negative (convention standard)
    sentiment_label = "Negative" if prediction == 1 else "Positive"
    
    return {
        'sentiment': sentiment_label,
        'confidence': confidence,
        'prob_positive': probabilities[0].item(),  # Inversé
        'prob_negative': probabilities[1].item()   # Inversé
    }

# Test avec la version corrigée
print("🔧 TEST AVEC LABELS CORRIGÉS:")
print("="*80)

for expected_sentiment, text in test_cases_diagnostic.items():
    result = analyze_sentiment_fixed(text, model, tokenizer, device)
    
    print(f"\n📝 {expected_sentiment} : {text[:60]}...")
    print(f"   Prédit : {result['sentiment']} (confiance: {result['confidence']:.2%})")
    print(f"   ✓" if result['sentiment'] == expected_sentiment else "   ❌")

🔧 TEST AVEC LABELS CORRIGÉS:

📝 Positive : Apple stock surges to record highs as earnings beat expectat...
   Prédit : Negative (confiance: 78.75%)
   ❌

📝 Negative : Company files for bankruptcy amid mounting debt crisis. Shar...
   Prédit : Positive (confiance: 96.03%)
   ❌


## 6. Exemples d'utilisation

In [29]:
# Exemple 1 : News générales sur la finance
df_finance = analyze_news_pipeline(
    query="stocks OR market OR trading",
    page_size=10
)

# Afficher les résultats
print("\n📊 RÉSULTATS")
print(df_finance[['title', 'sentiment', 'confidence']].head(10))

🔄 DÉBUT DU PIPELINE

📡 Récupération des news (query='stocks OR market OR trading')...
✓ 10 articles récupérés

🤖 Analyse de sentiment en cours...
   Analysé 5/10 articles...
   Analysé 10/10 articles...

✓ 10 news analysées avec succès

📊 RÉSULTATS
                                               title sentiment  confidence
0  NorthStar Achieves Commercial-Scale Ac-225 Pro...  Positive    0.834642
1  Trump won’t rule out military force to acquire...  Negative    0.941200
2  Letenda is building new 30-ft electric midibus...  Negative    0.577747
3  Joby Aviation to acquire manufacturing facilit...  Negative    0.597531
4  United Airlines price target raised by $13 at ...  Positive    0.576018
5  JetBlue price target raised by $1 at TD Cowen,...  Positive    0.547066
6  Delta Air Lines price target raised by $5 at T...  Positive    0.650701
7  Fullerton Health Deepens Specialty Care Capabi...  Positive    0.810705
8  CoinDesk 20 Performance Update: Uniswap (UNI) ...  Negative    0.937489
9

In [30]:
# Exemple 2 : News sur des entreprises spécifiques
df_tech = analyze_news_pipeline(
    query="Apple OR Tesla OR Microsoft OR Google",
    page_size=15
)

# Filtrer par confiance élevée (> 80%)
df_high_confidence = df_tech[df_tech['confidence'] > 0.8]

print(f"\n🎯 News avec confiance > 80% : {len(df_high_confidence)} articles")
print(df_high_confidence[['title', 'sentiment', 'confidence']])

🔄 DÉBUT DU PIPELINE

📡 Récupération des news (query='Apple OR Tesla OR Microsoft OR Google')...
✓ 15 articles récupérés

🤖 Analyse de sentiment en cours...
   Analysé 5/15 articles...
   Analysé 10/15 articles...
   Analysé 15/15 articles...

✓ 15 news analysées avec succès

🎯 News avec confiance > 80% : 4 articles
                                                title sentiment  confidence
0   Lutron's Smart Blinds Know Exactly When the Su...  Positive    0.838816
1   Lutron's Smart Blinds Know Exactly When the Su...  Positive    0.838816
12  Mamdani’s woke, white tenant advocate Cea Weav...  Negative    0.944135
13  I’m a trained TV calibrator, and here are the ...  Positive    0.843425


In [31]:
# Exemple 3 : Analyse par sentiment
print("\n📈 STATISTIQUES")
print(f"Sentiment positif : {(df_tech['sentiment'] == 'Positive').sum()} articles")
print(f"Sentiment négatif : {(df_tech['sentiment'] == 'Negative').sum()} articles")
print(f"\nConfiance moyenne : {df_tech['confidence'].mean():.2%}")


📈 STATISTIQUES
Sentiment positif : 9 articles
Sentiment négatif : 6 articles

Confiance moyenne : 69.01%


## 7. Export des résultats (pour intégration équipe)

In [32]:
# Sauvegarder en CSV
output_file = f"sentiment_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_tech.to_csv(output_file, index=False)
print(f"✓ Résultats sauvegardés dans : {output_file}")

# Ou en JSON pour API
output_json = output_file.replace('.csv', '.json')
df_tech.to_json(output_json, orient='records', indent=2)
print(f"✓ Résultats JSON dans : {output_json}")

✓ Résultats sauvegardés dans : sentiment_analysis_20260108_151914.csv
✓ Résultats JSON dans : sentiment_analysis_20260108_151914.json


## 8. Fonction API-ready pour votre équipe

In [33]:
def get_sentiment_analysis(query, max_results=20):
    """
    Fonction simple pour l'équipe backend/frontend
    
    Args:
        query: Requête de recherche
        max_results: Nombre max de résultats
    
    Returns:
        dict formaté pour API
    """
    df = analyze_news_pipeline(query=query, page_size=max_results)
    
    if df.empty:
        return {"status": "error", "message": "No news found", "data": []}
    
    return {
        "status": "success",
        "query": query,
        "total_articles": len(df),
        "positive_count": (df['sentiment'] == 'Positive').sum(),
        "negative_count": (df['sentiment'] == 'Negative').sum(),
        "average_confidence": float(df['confidence'].mean()),
        "articles": df.to_dict('records')
    }

# Test
result = get_sentiment_analysis("cryptocurrency", max_results=5)
print(f"\nStatus: {result['status']}")
print(f"Total: {result['total_articles']} articles")
print(f"Positif: {result['positive_count']}, Négatif: {result['negative_count']}")

🔄 DÉBUT DU PIPELINE

📡 Récupération des news (query='cryptocurrency')...


✓ 5 articles récupérés

🤖 Analyse de sentiment en cours...
   Analysé 5/5 articles...

✓ 5 news analysées avec succès

Status: success
Total: 5 articles
Positif: 2, Négatif: 3
